File Structure as follows on Google Collab:

*   content/
  *   {48 photos from 5025 dataset}












Implementation heavily influenced from Vladimir Iglovikov's pytorch implementation, retinaface: 

https://github.com/ternaus/retinaface

Download the two packages below and restart the runtime as prompted in the output

In [ ]:
!pip install -U ipykernel
!pip install modin[dask]

After runtime has been restarted, install retinaface_pytorch package

In [ ]:
!pip install -U retinaface_pytorch > /dev/null

In [ ]:
#Import necessary packages
import cv2
from google.colab.patches import cv2_imshow
import os
import json
from matplotlib import pyplot as plt
from retinaface.pre_trained_models import get_model
from retinaface.utils import vis_annotations
import numpy as np

In [ ]:
plt.rcParams["figure.figsize"] = (15, 15)

In [ ]:
#Instantiate the pretrained retinface model
model = get_model("resnet50_2020-07-20", max_size=(300))
model.eval()

In [ ]:

RetinaNet_annotated = {}
#Iterate through images and extract ID
for filename in os.listdir("/content/"):
  if filename.endswith(".jpg"):
    if (filename.split("_")[0][0] == '0'):
      id = int(filename.split("_")[0][1])
    else:
      id = int(filename.split("_")[0])

    #Import image in BGR colour mode and conver to RGB colour mode 
    img = cv2.imread(filename)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    #Call the prediction function of the model to output predictions for image
    annotation = model.predict_jsons(image)
    annotation = annotation[0]
    #Display the bounding box and landmarks on face
    x = (annotation["bbox"][0])
    y = (annotation["bbox"][1])
    w = (annotation["bbox"][2])-(annotation["bbox"][0])
    h = (annotation["bbox"][3])-(annotation["bbox"][1])
    # print(x,y,w,h)
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), 3)
    for x in annotation["landmarks"]:
      cv2.circle(img, (x[0], x[1]), 1, (0, 0, 255), -1)
    cv2_imshow(img) 
    bb = {}
    lm = {}
    #Place x,y,w,h bounding box values in dictionary
    bb = {"x":float(annotation["bbox"][0]),"y":float(annotation["bbox"][1]),"w":float(annotation["bbox"][2])-float(annotation["bbox"][0]),"h":float(annotation["bbox"][3])-float(annotation["bbox"][1])}
    #Place landmark coordinates in dictionary
    lm ={'left_eye': annotation["landmarks"][0], 'right_eye': annotation["landmarks"][1], 'nose': annotation["landmarks"][2], 'left_mouth': annotation["landmarks"][3], 'right_mouth': annotation["landmarks"][4]}
    #Add bounding box and landmark coordinates to dictionary for every face id
    RetinaNet_annotated[id]={"bounding_box":bb,"landmarks":lm}



In [ ]:
#Export dictionary as JSON file for further analysis
json = json.dumps(RetinaNet_annotated)
f = open("retinaface_annotated.json","w")
f.write(json)
f.close()